In [1]:
import os
import math
import sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
% matplotlib inline  
from PIL import Image
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from tensorboardX import SummaryWriter

import logging
import imp
imp.reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')
# logging.basicConfig(format='[%(levelname)s %(asctime)s %(filename)s:%(lineno)s] %(message)s', level=logging.INFO)

In [2]:
class MLTData():
    def __init__(self, batch_size=32, image_size=128, align_object=True, train_proportion=1):
        self.name = 'mlt'
        self.img_dir = '/media/yi/DATA/data-orig/MLT/image'
        self.box_dir = '/home/yi/code/few_shot/mlt/box'
        self.meta = self.get_meta(self.img_dir, self.box_dir)
 
        self.train_meta = self.assign_meta('train')
        self.test_meta = self.assign_meta('test')
        
        self.batch_size = batch_size
        self.im_size = image_size
        self.align_object= align_object
        self.train_proportion = train_proportion
        num_image = len(self.train_meta['img'])
        self.train_meta['img'] = self.train_meta['img'][0:int(num_image * train_proportion)]
        self.train_meta['box'] = self.train_meta['box'][0:int(num_image * train_proportion)]
        # print len(self.train_meta['img'])
        self.train_idx = np.random.permutation(len(self.train_meta['img']))
        self.train_cnt = 0
        self.test_idx = np.random.permutation(len(self.test_meta['img']))
        self.test_cnt = 0
        
        # print len(self.train_meta['img'])
        # print len(self.train_meta['box'])
        # print len(self.train_idx)
        # print len(self.test_meta['img'])
        # print len(self.test_meta['box'])
        print('number of training image: %d, number of testing image: %d' % 
              (len(self.train_meta['img']), len(self.test_meta['img'])))
        self.compute_statistics('train')
        self.compute_statistics('test')
        
        self.class_map = {3:0, 4:1, 5:2, 6:3, 13:4, 24:5}
        
    def get_meta(self, img_dir, box_dir):
        meta = {}
        meta['img'], meta['box'] = [], []
        for sub_dir in os.listdir(box_dir):
            box_files = os.listdir(os.path.join(box_dir, sub_dir))
            box_files.sort(key=lambda f: int(filter(str.isdigit, f)))
            box_file_names = [os.path.join(box_dir, sub_dir, f) for f in box_files]
            img_files = []
            for f in box_files:
                file_name, file_ext = os.path.splitext(f)
                img_files.append(file_name + '_color.jpg')
            img_file_names = [os.path.join(img_dir, sub_dir, f) for f in img_files]
            meta['img'].extend(img_file_names)
            meta['box'].extend(box_file_names)
        # print meta['img'][0:10], meta['box'][0:10], len(meta['img']), len(meta['box'])
        return meta
    
    def assign_meta(self, status='train'):
        num_image = len(self.meta['img'])
        if status == 'train':
            meta = {}
            meta['img'] = self.meta['img'][0:int(num_image * 0.9)]
            meta['box'] = self.meta['box'][0:int(num_image * 0.9)]
        else:
            meta = {}
            meta['img'] = self.meta['img'][int(num_image * 0.9):]
            meta['box'] = self.meta['box'][int(num_image * 0.9):]
        return meta
    
    def compute_statistics(self, status='train'):
        if status == 'train':
            box_meta = self.train_meta['box']
        else:
            box_meta = self.test_meta['box']
        box = []
        for box_file_name in box_meta:
            with open(box_file_name) as txt_file:
                box_info = txt_file.readlines()
            for row in box_info:
                row = row.strip().split(' ')
                box.append(row)
        box = np.array(box).astype(np.int)

        cls = box[:, 4]
        box = box[:, 0:4]
        cls_labels = set(list(cls))
        count = dict()
        max_count = 0
        total_count = 0
        for cls_label in cls_labels:
            count[cls_label] = (cls == cls_label).sum()
            if count[cls_label] > max_count:
                max_count = count[cls_label]
            total_count = total_count + count[cls_label]
        print(status)
        print(count)
        print(max_count * 1.0 / total_count)

    def get_next_batch(self, status='train'):
        batch_size, im_size = self.batch_size, self.im_size
        im = np.zeros((batch_size, im_size, im_size, 3))
        label = np.zeros(batch_size)
        i = 0
        while i < batch_size:
            if status == 'train':
                image_name = self.train_meta['img'][self.train_idx[self.train_cnt]]
                box_name = self.train_meta['box'][self.train_idx[self.train_cnt]]
                # print image_name
                # print box_name
            else:
                image_name = self.test_meta['img'][self.test_idx[self.test_cnt]]
                box_name = self.test_meta['box'][self.test_idx[self.test_cnt]]
            image = np.array(Image.open(image_name))
            # image = cv2.imread(image_name)
            # clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
            # image[:, :, 0] = clahe.apply(image[:, :, 0])
            # image[:, :, 1] = clahe.apply(image[:, :, 1])
            # image[:, :, 2] = clahe.apply(image[:, :, 2])
            image = image / 255.0
            box = []
            with open(box_name) as txt_file:
                box_info = txt_file.readlines()
            for row in box_info:
                row = row.strip().split(' ')
                box.append(row)
            box = np.array(box).astype(np.int)
            
            if status == 'train':
                self.train_cnt = self.train_cnt + 1
                if self.train_cnt >= len(self.train_meta['img']):
                    self.train_idx = np.random.permutation(len(self.train_meta['img']))
                    self.train_cnt = 0
            else:
                self.test_cnt = self.test_cnt + 1
                if self.test_cnt >= len(self.test_meta['img']):
                    self.test_idx = np.random.permutation(len(self.test_meta['img']))
                    self.test_cnt = 0
            
            for n in range(box.shape[0]):
                cls = box[n, 4]
                bb = box[n, 0:4]
                crop_im = self.crop_image(image, bb, im_size)
                im[i, :, :, :] = cv2.resize(crop_im, (im_size, im_size), interpolation=cv2.INTER_AREA)
                label[i] = self.class_map[cls]
                i = i + 1
                if i >= batch_size:
                    break
        im = im.transpose((0, 3, 1, 2))
        return im, label
    
    def crop_image(self, image, bb, im_size):
        im_height, im_width = image.shape[0], image.shape[1]
        x_c, y_c = (bb[0] + bb[2]) / 2, (bb[1] + bb[3]) / 2
        width, height = bb[2] - bb[0], bb[3] - bb[1]
        if self.align_object:
            height = height * 2
            width = width * 2
        else:
            height = np.random.randint(height * 2, height * 5)
            width = np.random.randint(width * 2, width * 5)
            
        y_s = max(0, y_c - height / 2)
        y_e = min(im_height, y_c + height / 2)
        x_s = max(0, x_c - width / 2)
        x_e = min(im_width, x_c + width / 2)
        im = image[y_s:y_e, x_s:x_e, :]
        return im
    
    def display(self, im, label):
        img = im[0, :, :, :].transpose(1, 2, 0)
        
        plt.figure()
        plt.imshow(img)
        plt.axis('off')
        plt.show()
        
        # print(img)
        print(label[0])

In [3]:
class BaseNet(nn.Module):
    def __init__(self, im_height, im_width, im_channel, n_class):
        super(BaseNet, self).__init__()
        num_hidden = 32
        self.bn0 = nn.BatchNorm2d(im_channel)
        self.conv1 = nn.Conv2d(im_channel, num_hidden, 3, 1, 1)
        self.bn1 = nn.BatchNorm2d(num_hidden)
        self.conv2 = nn.Conv2d(num_hidden, num_hidden*2, 3, 1, 1)
        self.bn2 = nn.BatchNorm2d(num_hidden*2)
        self.conv3 = nn.Conv2d(num_hidden*2, num_hidden*4, 3, 1, 1)
        self.bn3 = nn.BatchNorm2d(num_hidden*4)
        self.conv4 = nn.Conv2d(num_hidden*4, num_hidden*8, 3, 1, 1)
        self.bn4 = nn.BatchNorm2d(num_hidden*8)
        self.conv5 = nn.Conv2d(num_hidden*8, num_hidden*16, 3, 1, 1)
        self.bn5 = nn.BatchNorm2d(num_hidden*16)
        self.conv6 = nn.Conv2d(num_hidden*16, num_hidden*32, 3, 1, 1)
        self.bn6 = nn.BatchNorm2d(num_hidden*32)
        self.fc = nn.Linear(num_hidden*32, n_class)

        self.maxpool = nn.MaxPool2d(2, stride=2, return_indices=False, ceil_mode=False)
        self.avgpool = nn.AvgPool2d(im_height / 32, stride=1)

    def forward(self, im_input):
        x = self.bn0(im_input)
        x1 = F.relu(self.bn1(self.conv1(x)))
        x2 = self.maxpool(x1)
        x2 = F.relu(self.bn2(self.conv2(x2)))
        x3 = self.maxpool(x2)
        x3 = F.relu(self.bn3(self.conv3(x3)))
        x4 = self.maxpool(x3)
        x4 = F.relu(self.bn4(self.conv4(x4)))
        x5 = self.maxpool(x4)
        x5 = F.relu(self.bn5(self.conv5(x5)))
        x6 = self.maxpool(x5)
        x6 = F.relu(self.bn6(self.conv6(x6)))
        x7 = self.avgpool(x6)
        x7 = x7.view(x7.size(0), -1)
        pred = self.fc(x7)
        return pred

In [4]:
class BaseDemo(object):
    def __init__(self, learning_rate, train_epoch, test_epoch, test_interval, save_interval, save_dir, batch_size, 
                 image_size, image_channel, num_class, tensorboard_path, init_model_path, align_object, 
                 train_proportion=1):
        self.learning_rate = learning_rate
        self.train_epoch = train_epoch
        self.test_epoch = test_epoch
        self.test_interval = test_interval
        self.save_interval = save_interval
        self.save_dir = save_dir
        self.best_test_acc = -1e10
        self.batch_size = batch_size
        self.im_size = image_size
        self.im_channel = image_channel
        self.n_class = num_class
        self.tensorboard_path = tensorboard_path
        self.init_model_path = init_model_path
        self.data = MLTData(batch_size, image_size, align_object, train_proportion)
        self.model = self.init_model()

    def init_model(self):
        self.model = BaseNet(self.im_size, self.im_size, self.im_channel, self.n_class)
        if torch.cuda.is_available():
            # model = torch.nn.DataParallel(model).cuda()
            self.model = self.model.cuda()
        if self.init_model_path is not '':
            self.model.load_state_dict(torch.load(self.init_model_path))
        return self.model

    def train(self):
        writer = SummaryWriter(self.tensorboard_path)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        train_loss_all, train_acc_all = [], []
        for epoch in range(self.train_epoch):
            optimizer.zero_grad()
            im, label = self.data.get_next_batch('train')
            im = Variable(torch.from_numpy(im).float())
            label = Variable(torch.from_numpy(label).long())
            if torch.cuda.is_available():
                im, label = im.cuda(), label.cuda()
            pred = self.model(im)
            loss = criterion(pred, label)
            loss.backward()
            optimizer.step()

            _, pred_label = torch.max(pred.data, 1)
            acc = (pred_label == label.data).sum() * 1.0 / label.size(0)
            
            writer.add_scalar('train_loss', loss.data[0], epoch)
            train_loss_all.append(loss.data[0])
            if len(train_loss_all) > 100:
                train_loss_all.pop(0)
            ave_train_loss = sum(train_loss_all) / float(len(train_loss_all))
            logging.info('epoch %d, train loss: %.2f, average train loss: %.2f', epoch, loss.data[0], ave_train_loss)
            writer.add_scalar('train_acc', acc, epoch)
            train_acc_all.append(acc)
            if len(train_acc_all) > 100:
                train_acc_all.pop(0)
            ave_train_acc = sum(train_acc_all) / float(len(train_acc_all))
            logging.info('epoch %d, train accuracy: %.2f, average train accuracy: %.2f', epoch, acc, ave_train_acc)
            if (epoch+1) % self.save_interval == 0:
                logging.info('epoch %d, saving model', epoch)
                with open(os.path.join(self.save_dir, '%d.pth' % epoch), 'w') as handle:
                    torch.save(self.model.state_dict(), handle)
            if (epoch+1) % self.test_interval == 0:
                logging.info('epoch %d, testing', epoch)
                test_loss, test_acc = self.validate()
                writer.add_scalar('test_loss', test_loss, epoch)
                writer.add_scalar('test_acc', test_acc, epoch)
        writer.close()

    def validate(self):
        test_loss, test_acc = self.test()
        if test_acc >= self.best_test_acc:
            logging.info('model save to %s', os.path.join(self.save_dir, 'model.pth'))
            with open(os.path.join(self.save_dir, 'model.pth'), 'w') as handle:
                torch.save(self.model.state_dict(), handle)
            self.best_test_acc = test_acc
        logging.info('current best test accuracy: %.2f', self.best_test_acc)
        return test_loss, test_acc

    def test(self):
        test_loss_all, test_acc_all = [], []
        criterion = nn.CrossEntropyLoss()
        for epoch in range(self.test_epoch):
            im, label = self.data.get_next_batch('test')
            im = Variable(torch.from_numpy(im).float(), volatile=True)
            label = Variable(torch.from_numpy(label).long(), volatile=True)
            if torch.cuda.is_available():
                im, label = im.cuda(), label.cuda()
            pred = self.model(im)
            loss = criterion(pred, label)
            _, pred_label = torch.max(pred.data, 1)
            acc = (pred_label == label.data).sum() * 1.0 / label.size(0)

            test_loss_all.append(loss.data[0])
            if len(test_loss_all) > 100:
                test_loss_all.pop(0)
            test_acc_all.append(acc)
            if len(test_acc_all) > 100:
                test_acc_all.pop(0)

        test_loss = np.mean(np.array(test_loss_all))
        test_acc = np.mean(np.array(test_acc_all))
        logging.info('average test loss: %.2f', test_loss)
        logging.info('average test accuracy: %.2f', test_acc)
        return test_loss, test_acc

In [5]:
command = 'train'

learning_rate = 1e-3
train_epoch = 800
test_epoch = 10
test_interval = 100
save_interval = 1001
save_dir = './'
batch_size = 64
im_size = 128
im_channel = 3
n_class = 6
tensorboard_path = './tensorboard/aligned_100'
init_model_path = ''
align_object = True
train_proportion = 1
demo = BaseDemo(learning_rate, train_epoch, test_epoch, test_interval, save_interval, save_dir, batch_size, im_size, 
                im_channel, n_class, tensorboard_path, init_model_path, align_object, train_proportion)
if command == 'train':
    demo.train()
elif command == 'test':
    demo.test()

number of training image: 6993, number of testing image: 777
train
{3: 2219, 4: 3405, 5: 2356, 6: 2147, 13: 1678, 24: 1070}
0.2644660194174757
test
{3: 264, 4: 425, 5: 271, 6: 216, 13: 150, 24: 111}
0.29575504523312457


03:25:11 INFO:epoch 0, train loss: 1.76, average train loss: 1.76
03:25:11 INFO:epoch 0, train accuracy: 0.27, average train accuracy: 0.27
03:25:12 INFO:epoch 1, train loss: 1.95, average train loss: 1.85
03:25:12 INFO:epoch 1, train accuracy: 0.17, average train accuracy: 0.22
03:25:12 INFO:epoch 2, train loss: 1.85, average train loss: 1.85
03:25:12 INFO:epoch 2, train accuracy: 0.25, average train accuracy: 0.23
03:25:13 INFO:epoch 3, train loss: 1.91, average train loss: 1.87
03:25:13 INFO:epoch 3, train accuracy: 0.30, average train accuracy: 0.25
03:25:13 INFO:epoch 4, train loss: 1.67, average train loss: 1.83
03:25:13 INFO:epoch 4, train accuracy: 0.28, average train accuracy: 0.25
03:25:14 INFO:epoch 5, train loss: 1.63, average train loss: 1.79
03:25:14 INFO:epoch 5, train accuracy: 0.33, average train accuracy: 0.27
03:25:14 INFO:epoch 6, train loss: 1.70, average train loss: 1.78
03:25:14 INFO:epoch 6, train accuracy: 0.20, average train accuracy: 0.26
03:25:15 INFO:epoch 

In [6]:
command = 'train'

learning_rate = 1e-3
train_epoch = 800
test_epoch = 10
test_interval = 100
save_interval = 1001
save_dir = './'
batch_size = 64
im_size = 128
im_channel = 3
n_class = 6
tensorboard_path = './tensorboard/unaligned_100'
init_model_path = ''
align_object = False
train_proportion = 1
demo = BaseDemo(learning_rate, train_epoch, test_epoch, test_interval, save_interval, save_dir, batch_size, im_size, 
                im_channel, n_class, tensorboard_path, init_model_path, align_object, train_proportion)
if command == 'train':
    demo.train()
elif command == 'test':
    demo.test()

number of training image: 6993, number of testing image: 777
train
{3: 2219, 4: 3405, 5: 2356, 6: 2147, 13: 1678, 24: 1070}
0.2644660194174757
test
{3: 264, 4: 425, 5: 271, 6: 216, 13: 150, 24: 111}
0.29575504523312457


03:32:10 INFO:epoch 0, train loss: 1.77, average train loss: 1.77
03:32:10 INFO:epoch 0, train accuracy: 0.27, average train accuracy: 0.27
03:32:10 INFO:epoch 1, train loss: 1.90, average train loss: 1.84
03:32:10 INFO:epoch 1, train accuracy: 0.19, average train accuracy: 0.23
03:32:11 INFO:epoch 2, train loss: 1.79, average train loss: 1.82
03:32:11 INFO:epoch 2, train accuracy: 0.31, average train accuracy: 0.26
03:32:11 INFO:epoch 3, train loss: 1.94, average train loss: 1.85
03:32:11 INFO:epoch 3, train accuracy: 0.11, average train accuracy: 0.22
03:32:12 INFO:epoch 4, train loss: 1.85, average train loss: 1.85
03:32:12 INFO:epoch 4, train accuracy: 0.20, average train accuracy: 0.22
03:32:12 INFO:epoch 5, train loss: 1.83, average train loss: 1.85
03:32:12 INFO:epoch 5, train accuracy: 0.22, average train accuracy: 0.22
03:32:13 INFO:epoch 6, train loss: 1.77, average train loss: 1.84
03:32:13 INFO:epoch 6, train accuracy: 0.22, average train accuracy: 0.22
03:32:13 INFO:epoch 

In [7]:
command = 'train'

learning_rate = 1e-3
train_epoch = 800
test_epoch = 100
test_interval = 100
save_interval = 1001
save_dir = './'
batch_size = 64
im_size = 128
im_channel = 3
n_class = 6
tensorboard_path = './tensorboard/aligned_50'
init_model_path = ''
align_object = True
train_proportion = 0.5
demo = BaseDemo(learning_rate, train_epoch, test_epoch, test_interval, save_interval, save_dir, batch_size, im_size, 
                im_channel, n_class, tensorboard_path, init_model_path, align_object, train_proportion)
if command == 'train':
    demo.train()
elif command == 'test':
    demo.test()

number of training image: 3496, number of testing image: 777
train
{3: 1092, 4: 1622, 5: 1249, 6: 1120, 13: 824, 24: 525}
0.25217661691542287
test
{3: 264, 4: 425, 5: 271, 6: 216, 13: 150, 24: 111}
0.29575504523312457


03:39:32 INFO:epoch 0, train loss: 1.81, average train loss: 1.81
03:39:32 INFO:epoch 0, train accuracy: 0.14, average train accuracy: 0.14
03:39:32 INFO:epoch 1, train loss: 2.10, average train loss: 1.95
03:39:32 INFO:epoch 1, train accuracy: 0.16, average train accuracy: 0.15
03:39:33 INFO:epoch 2, train loss: 2.18, average train loss: 2.03
03:39:33 INFO:epoch 2, train accuracy: 0.17, average train accuracy: 0.16
03:39:33 INFO:epoch 3, train loss: 1.96, average train loss: 2.01
03:39:33 INFO:epoch 3, train accuracy: 0.17, average train accuracy: 0.16
03:39:34 INFO:epoch 4, train loss: 1.82, average train loss: 1.97
03:39:34 INFO:epoch 4, train accuracy: 0.19, average train accuracy: 0.17
03:39:34 INFO:epoch 5, train loss: 1.62, average train loss: 1.91
03:39:34 INFO:epoch 5, train accuracy: 0.20, average train accuracy: 0.17
03:39:35 INFO:epoch 6, train loss: 1.70, average train loss: 1.88
03:39:35 INFO:epoch 6, train accuracy: 0.34, average train accuracy: 0.20
03:39:35 INFO:epoch 

In [8]:
command = 'train'

learning_rate = 1e-3
train_epoch = 800
test_epoch = 100
test_interval = 100
save_interval = 1001
save_dir = './'
batch_size = 64
im_size = 128
im_channel = 3
n_class = 6
tensorboard_path = './tensorboard/unaligned_50'
init_model_path = ''
align_object = False
train_proportion = 0.5
demo = BaseDemo(learning_rate, train_epoch, test_epoch, test_interval, save_interval, save_dir, batch_size, im_size, 
                im_channel, n_class, tensorboard_path, init_model_path, align_object, train_proportion)
if command == 'train':
    demo.train()
elif command == 'test':
    demo.test()

number of training image: 3496, number of testing image: 777
train
{3: 1092, 4: 1622, 5: 1249, 6: 1120, 13: 824, 24: 525}
0.25217661691542287
test
{3: 264, 4: 425, 5: 271, 6: 216, 13: 150, 24: 111}
0.29575504523312457


03:51:25 INFO:epoch 0, train loss: 1.89, average train loss: 1.89
03:51:25 INFO:epoch 0, train accuracy: 0.02, average train accuracy: 0.02
03:51:25 INFO:epoch 1, train loss: 2.03, average train loss: 1.96
03:51:25 INFO:epoch 1, train accuracy: 0.06, average train accuracy: 0.04
03:51:26 INFO:epoch 2, train loss: 1.81, average train loss: 1.91
03:51:26 INFO:epoch 2, train accuracy: 0.20, average train accuracy: 0.09
03:51:26 INFO:epoch 3, train loss: 1.84, average train loss: 1.90
03:51:26 INFO:epoch 3, train accuracy: 0.23, average train accuracy: 0.13
03:51:27 INFO:epoch 4, train loss: 1.77, average train loss: 1.87
03:51:27 INFO:epoch 4, train accuracy: 0.28, average train accuracy: 0.16
03:51:27 INFO:epoch 5, train loss: 1.55, average train loss: 1.82
03:51:27 INFO:epoch 5, train accuracy: 0.39, average train accuracy: 0.20
03:51:28 INFO:epoch 6, train loss: 1.92, average train loss: 1.83
03:51:28 INFO:epoch 6, train accuracy: 0.19, average train accuracy: 0.20
03:51:28 INFO:epoch 

In [9]:
command = 'train'

learning_rate = 1e-3
train_epoch = 800
test_epoch = 100
test_interval = 100
save_interval = 1001
save_dir = './'
batch_size = 64
im_size = 128
im_channel = 3
n_class = 6
tensorboard_path = './tensorboard/aligned_20'
init_model_path = ''
align_object = True
train_proportion = 0.2
demo = BaseDemo(learning_rate, train_epoch, test_epoch, test_interval, save_interval, save_dir, batch_size, im_size, 
                im_channel, n_class, tensorboard_path, init_model_path, align_object, train_proportion)
if command == 'train':
    demo.train()
elif command == 'test':
    demo.test()

number of training image: 1398, number of testing image: 777
train
{3: 420, 4: 667, 5: 484, 6: 452, 13: 313, 24: 195}
0.26353220071118133
test
{3: 264, 4: 425, 5: 271, 6: 216, 13: 150, 24: 111}
0.29575504523312457


04:04:03 INFO:epoch 0, train loss: 1.76, average train loss: 1.76
04:04:03 INFO:epoch 0, train accuracy: 0.25, average train accuracy: 0.25
04:04:04 INFO:epoch 1, train loss: 1.62, average train loss: 1.69
04:04:04 INFO:epoch 1, train accuracy: 0.38, average train accuracy: 0.31
04:04:04 INFO:epoch 2, train loss: 1.94, average train loss: 1.77
04:04:04 INFO:epoch 2, train accuracy: 0.25, average train accuracy: 0.29
04:04:05 INFO:epoch 3, train loss: 1.71, average train loss: 1.76
04:04:05 INFO:epoch 3, train accuracy: 0.34, average train accuracy: 0.30
04:04:05 INFO:epoch 4, train loss: 1.65, average train loss: 1.73
04:04:05 INFO:epoch 4, train accuracy: 0.28, average train accuracy: 0.30
04:04:06 INFO:epoch 5, train loss: 1.58, average train loss: 1.71
04:04:06 INFO:epoch 5, train accuracy: 0.34, average train accuracy: 0.31
04:04:06 INFO:epoch 6, train loss: 1.59, average train loss: 1.69
04:04:06 INFO:epoch 6, train accuracy: 0.30, average train accuracy: 0.31
04:04:07 INFO:epoch 

In [10]:
command = 'train'

learning_rate = 1e-3
train_epoch = 800
test_epoch = 100
test_interval = 100
save_interval = 1001
save_dir = './'
batch_size = 64
im_size = 128
im_channel = 3
n_class = 6
tensorboard_path = './tensorboard/unaligned_20'
init_model_path = ''
align_object = False
train_proportion = 0.2
demo = BaseDemo(learning_rate, train_epoch, test_epoch, test_interval, save_interval, save_dir, batch_size, im_size, 
                im_channel, n_class, tensorboard_path, init_model_path, align_object, train_proportion)
if command == 'train':
    demo.train()
elif command == 'test':
    demo.test()

number of training image: 1398, number of testing image: 777
train
{3: 420, 4: 667, 5: 484, 6: 452, 13: 313, 24: 195}
0.26353220071118133
test
{3: 264, 4: 425, 5: 271, 6: 216, 13: 150, 24: 111}
0.29575504523312457


04:15:58 INFO:epoch 0, train loss: 1.81, average train loss: 1.81
04:15:58 INFO:epoch 0, train accuracy: 0.16, average train accuracy: 0.16
04:15:59 INFO:epoch 1, train loss: 2.19, average train loss: 2.00
04:15:59 INFO:epoch 1, train accuracy: 0.22, average train accuracy: 0.19
04:15:59 INFO:epoch 2, train loss: 1.96, average train loss: 1.99
04:15:59 INFO:epoch 2, train accuracy: 0.12, average train accuracy: 0.17
04:16:00 INFO:epoch 3, train loss: 1.77, average train loss: 1.93
04:16:00 INFO:epoch 3, train accuracy: 0.31, average train accuracy: 0.20
04:16:00 INFO:epoch 4, train loss: 1.74, average train loss: 1.89
04:16:00 INFO:epoch 4, train accuracy: 0.28, average train accuracy: 0.22
04:16:01 INFO:epoch 5, train loss: 1.87, average train loss: 1.89
04:16:01 INFO:epoch 5, train accuracy: 0.20, average train accuracy: 0.22
04:16:01 INFO:epoch 6, train loss: 1.86, average train loss: 1.88
04:16:01 INFO:epoch 6, train accuracy: 0.20, average train accuracy: 0.21
04:16:02 INFO:epoch 